In [ ]:
from experiments.attn.victory import *

In [ ]:
results = {}
for boardsize in (3, 5, 7, 9):
    for depth in [1, 2, 4, 8, 16]:
        for width in [8, 16, 32, 64, 128]:
            print(boardsize, depth, width)
            results[boardsize, depth, width] = run_trial(boardsize=boardsize, n_layers=depth, D=width)

In [ ]:
%debug

In [ ]:
outputs = model(b.worlds.obs)

In [ ]:
targets = terminal_actions(b.worlds)
outputs = model(b.worlds.obs)

infs = torch.full_like(targets, -np.inf)
loss = -outputs.reshape(B, -1).where(targets == 1., infs).max(-1).values

bad = loss.argsort().cpu().numpy()[::-1]

In [ ]:
plot_soln(bad[-2], b.worlds.obs, targets, outputs, model.attns)

In [ ]:
from rebar.recording import Encoder

i = bad[-2]
obs = b.worlds.obs[i].detach().cpu().numpy()
attn = model.attns[i].detach().cpu().numpy()
boardsize = int(attn.shape[1]**.5)
attn = attn.transpose(0, 2, 1).reshape(attn.shape[0], attn.shape[2], boardsize, boardsize)

rows, cols = attn.shape[:2]


with Encoder(fps=1) as enc:
    for r in range(rows+3):
        r = min(r, rows-1)
        fig, axes = plt.subplots(1, 1, squeeze=False)
        fig.set_size_inches(5, 5)

        colors = color_obs(obs)
        colors = np.concatenate([colors, attn[r].max(0)[..., None]], -1)
        
        plot_board(colors, ax=axes[0, 0], edges=False)
        enc(fig)
        plt.close(fig)
enc.notebook(height=320)

In [ ]:


color_obs(obs)